# 로컬 예측 모델

In [1]:
from glob import glob
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import cv2
import pickle
import warnings
warnings.filterwarnings('ignore')

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
import time
import albumentations as albu
from albumentations.pytorch import ToTensorV2

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances
from lightgbm import LGBMClassifier

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
all_img_path = np.array(sorted(glob('../data/train/*.png')))
test_img_path = np.array(sorted(glob('../data/test/*.png')))

In [3]:
train_y = pd.read_csv("../data/train_df.csv")

train_classes = train_y["class"]

class_unique = sorted(np.unique(train_classes))
class_unique = {key:value for key,value in zip(class_unique, range(len(class_unique)))}
class_decoder = {val:key for key, val in class_unique.items()}

all_classes = np.array([class_unique[k] for k in train_classes])

In [4]:
train_y = pd.read_csv("../data/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

all_label = np.array([label_unique[k] for k in train_labels])

In [5]:
def custom_metric(truth, predictions):
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macrF1', f1, True) 

def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (256, 256)).flatten()
    return img

train_imgs = np.array([img_load(m) for m in tqdm(all_img_path)])
test_imgs = np.array([img_load(n) for n in tqdm(test_img_path)])

train_imgs = train_imgs / 255
test_imgs = test_imgs / 255

  0%|          | 0/4277 [00:00<?, ?it/s]

  0%|          | 0/2154 [00:00<?, ?it/s]

In [6]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':4,
    'LEARNING_RATE':1e-2,
    'BATCH_SIZE':8,
    'SEED':42
}

rows_train = len(train_imgs) # 주어진 train data의 row 수
rows_test = len(test_imgs) # 주어진 test data의 row 수
# rows_train = len(all_img_path) # 주어진 train data의 row 수
# rows_test = len(test_img_path) # 주어진 test data의 row 수
num_trial = 100 # 파라미터 튜닝을 몇 번 진행하는지의 수
splits_hp = 3 # 파라미터 튜닝을 진행할 때의 kfold 수
splits_tr = 4 # 모델 트레이닝을 진행할 때의 kfold 수
basic_seed = 42 # default seed
num_seed_tr = 10 # 트레이닝 seed 개수
sel_seed = 3 # 선택할 seed 개수
num_classes = 88

pred_dict = {}
pred_test_dict = {}

In [34]:
all_label = pd.Series(all_label).astype('category')

In [ ]:
def lgb_objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": basic_seed,
        "verbosity": -1,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
#         "device": "gpu",
        "num_class": num_classes,
        "learning_rate": trial.suggest_uniform("learning_rate", 1e-2, 1e-1), # default=0.1, range=[0,1]
        "max_depth": trial.suggest_int("max_depth", 4, 8), # default=-1
        "reg_alpha": trial.suggest_loguniform("reg_alpha", 1e-2, 1e+1), # default=0
        "reg_lambda": trial.suggest_loguniform("reg_lambda", 1e-2, 1e+1), # default=0
        "num_leaves": trial.suggest_int("num_leaves", 31, 2000), # default=31, range=(1,130172]
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.2, 1.0), # feature_fraction, default=1
        "subsample": trial.suggest_uniform("subsample", 0.2, 1.0), # bagging_fraction, default=1, range=[0,1]
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 20), # bagging_freq, default=0
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 40), # min_data_in_leaf, default=20 
#         "max_bin": trial.suggest_int("max_bin", 100, 400),
    }
    
    kfold = StratifiedKFold(n_splits=splits_hp, random_state=basic_seed, shuffle=True)
    cv = np.zeros((rows_train, num_classes))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_imgs, all_label)):

        x_train, x_val = train_imgs[train_idx], train_imgs[val_idx]
        y_train, y_val = all_label[train_idx], all_label[val_idx]

        lgbmodel = LGBMClassifier(**params_lgb)
                                                                                        # 진행상황 보고싶을때 -1을 100으로
        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=200, verbose=30, eval_metric=custom_metric) 
        cv[val_idx, :] = lgbmodel.predict_proba(x_val)
        
#         print('fold', n+1, 'f1_score:', score_function(y_val, cv[val_idx, :].argmax(1)))
        
#     print('f1_score:', score_function(all_label, np.argmax(cv, axis=1)))
    return score_function(all_label, np.argmax(cv, axis=1))

In [ ]:
sampler = TPESampler(seed=basic_seed)
lgb_study = optuna.create_study(study_name="lgb_parameter_opt", direction="maximize", sampler=sampler)
lgb_study.optimize(lgb_objective, n_trials=num_trial)

lgb_best_hyperparams = lgb_study.best_trial.params
lgb_base_hyperparams = {'objective':'multiclass', 'n_estimators':10000, "num_class": num_classes, 
                        'lambda_l1':lgb_best_hyperparams['reg_alpha'],
                        'lambda_l2':lgb_best_hyperparams['reg_lambda'],
                        'reg_alpha':None, 'reg_lambda':None
                       }
lgb_best_hyperparams.update(lgb_base_hyperparams)
print("The best hyperparameters are:\n", lgb_best_hyperparams)
with open('../pickle/lgb_best_hyperparams.pickle', 'wb') as fw:
    pickle.dump(lgb_best_hyperparams, fw)

In [ ]:
optuna.visualization.matplotlib.plot_param_importances(lgb_study);

In [ ]:
optuna.visualization.matplotlib.plot_slice(lgb_study);

In [8]:
with open('../pickle/lgb_best_hyperparams.pickle', 'rb') as fw:
    lgb_best_hyperparams = pickle.load(fw)

In [10]:
pred_dict

{'lgb452': array([[1.10073597e-04, 8.33608628e-05, 1.01125931e-04, ...,
         4.45876348e-05, 7.65392523e-05, 8.83422371e-05],
        [2.75647268e-05, 2.36852120e-05, 3.00717816e-05, ...,
         1.07865431e-05, 2.21904418e-05, 2.62071409e-05],
        [3.12194834e-04, 2.99542594e-04, 2.65148572e-04, ...,
         3.52129214e-04, 2.32698079e-04, 2.91451488e-04],
        ...,
        [7.16435572e-04, 8.52659211e-04, 8.49488763e-04, ...,
         7.58827148e-04, 8.19252391e-04, 4.54377519e-04],
        [2.27709072e-04, 2.02499141e-04, 2.84418114e-04, ...,
         3.49181229e-03, 4.62815526e-04, 2.11499152e-04],
        [8.26567564e-04, 8.48735489e-04, 8.88222975e-04, ...,
         8.19595513e-04, 7.58519280e-04, 7.35531417e-04]]),
 'lgb226': array([[3.98766877e-05, 3.23916366e-05, 2.40799245e-05, ...,
         2.17610666e-05, 2.71788659e-05, 2.49859324e-05],
        [4.84891287e-05, 2.99203914e-05, 4.22698874e-05, ...,
         3.81993655e-05, 4.77097677e-05, 4.38602934e-05],
     

In [9]:
lucky_seeds = np.random.randint(0, 1000, num_seed_tr)

for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=splits_tr, random_state=seed, shuffle=True) # CV 늘려가면서 하기
    cv = np.zeros((rows_train, num_classes))
    pred_test = np.zeros((rows_test, num_classes))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_imgs, all_label)):
        
        x_train, x_val = train_imgs[train_idx], train_imgs[val_idx]
        y_train, y_val = all_label[train_idx], all_label[val_idx]

        lgbmodel = LGBMClassifier(**lgb_best_hyperparams)
                                                                                        # 진행상황 보고싶을때 -1을 100으로
        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=200, verbose=30, eval_metric=custom_metric) 
        cv[val_idx, :] = lgbmodel.predict_proba(x_val)
        pred_test += lgbmodel.predict_proba(test_imgs) / splits_tr
        print('fold', n+1, 'f1 score :', score_function(y_val, cv[val_idx, :].argmax(1)))
        
    pred_dict['lgb'+str(seed)] = cv
    pred_test_dict['lgb'+str(seed)] = pred_test
    print(f'seed {seed}', 'f1 score :', score_function(all_label, np.argmax(cv, axis=1)))

[30]	valid_0's multi_logloss: 1.26766	valid_0's macrF1: 0.150376
[60]	valid_0's multi_logloss: 1.11255	valid_0's macrF1: 0.157372
[90]	valid_0's multi_logloss: 1.06944	valid_0's macrF1: 0.152294
[120]	valid_0's multi_logloss: 1.05737	valid_0's macrF1: 0.157902
[150]	valid_0's multi_logloss: 1.05053	valid_0's macrF1: 0.158099
[180]	valid_0's multi_logloss: 1.04385	valid_0's macrF1: 0.158833
[210]	valid_0's multi_logloss: 1.04106	valid_0's macrF1: 0.159105
[240]	valid_0's multi_logloss: 1.0402	valid_0's macrF1: 0.15893
[270]	valid_0's multi_logloss: 1.03876	valid_0's macrF1: 0.158754
[300]	valid_0's multi_logloss: 1.03845	valid_0's macrF1: 0.158931
[330]	valid_0's multi_logloss: 1.03784	valid_0's macrF1: 0.158931
[360]	valid_0's multi_logloss: 1.0366	valid_0's macrF1: 0.159108
[390]	valid_0's multi_logloss: 1.0348	valid_0's macrF1: 0.159208
[420]	valid_0's multi_logloss: 1.03531	valid_0's macrF1: 0.159208
[450]	valid_0's multi_logloss: 1.03534	valid_0's macrF1: 0.159388
[480]	valid_0's m

[270]	valid_0's multi_logloss: 1.01375	valid_0's macrF1: 0.19678
[300]	valid_0's multi_logloss: 1.01393	valid_0's macrF1: 0.19678
[330]	valid_0's multi_logloss: 1.01271	valid_0's macrF1: 0.19678
[360]	valid_0's multi_logloss: 1.01103	valid_0's macrF1: 0.19678
[390]	valid_0's multi_logloss: 1.01197	valid_0's macrF1: 0.197044
[420]	valid_0's multi_logloss: 1.0122	valid_0's macrF1: 0.197044
[450]	valid_0's multi_logloss: 1.01237	valid_0's macrF1: 0.197044
[480]	valid_0's multi_logloss: 1.01237	valid_0's macrF1: 0.196875
[510]	valid_0's multi_logloss: 1.01095	valid_0's macrF1: 0.196875
[540]	valid_0's multi_logloss: 1.01051	valid_0's macrF1: 0.197212
[570]	valid_0's multi_logloss: 1.01052	valid_0's macrF1: 0.197212
fold 2 f1 score : 0.19714344748662513
[30]	valid_0's multi_logloss: 1.24051	valid_0's macrF1: 0.168056
[60]	valid_0's multi_logloss: 1.07692	valid_0's macrF1: 0.16691
[90]	valid_0's multi_logloss: 1.03894	valid_0's macrF1: 0.176635
[120]	valid_0's multi_logloss: 1.03097	valid_0'

KeyboardInterrupt: 

In [28]:
all_label = pd.Series(all_label).astype('int')

In [35]:
def sort_dict(model, pred_dict, pred_test_dict):
    pred_dict_local = {}
    for key, value in pred_dict.items():
        if model in key:
            pred_dict_local[key]=value

    pred_test_dict_local = {}
    for key, value in pred_test_dict.items():
        if model in key:
            pred_test_dict_local[key]=value

    pred_dict_new_local = dict(sorted(
        pred_dict_local.items(), 
        key=lambda x:score_function((all_label), np.argmax(list(x[1]), axis=1)), reverse=False)[:5])
    pred_test_dict_new_local = {}
    for key, value in pred_dict_new_local.items():
        pred_test_dict_new_local[key]=pred_test_dict_local[key]
        
    return pred_dict_new_local, pred_test_dict_new_local

In [37]:
def save_dict(model, pred_dict, pred_test_dict):
    with open('../pickle/pred_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_dict, fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_test_dict, fw)

In [38]:
pred_dict_lgb, pred_test_dict_lgb = sort_dict('lgb', pred_dict, pred_test_dict)
save_dict('lgb', pred_dict_lgb, pred_test_dict_lgb)

In [46]:
def load_dict(model):
    with open('../pickle/pred_dict_'+model+'.pickle', 'rb') as fw:
        pred_dict_new_local = pickle.load(fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'rb') as fw:
        pred_test_dict_new_local = pickle.load(fw)
    return pred_dict_new_local, pred_test_dict_new_local

In [47]:
pred_dict_lgb, pred_test_dict_lgb = load_dict('lgb')

In [ ]:
class LocalDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True):
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        img = cv2.imread(img_path)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [ ]:
splits_tr = 4
num_classes = 15
basic_seed = 42
rows_train = len(all_img_path) # 주어진 train data의 row 수
rows_test = len(test_img_path) # 주어진 test data의 row 수

In [ ]:
def train_local(model, optimizer, train_loader, vali_loader, device):
    model.to(device)

    # Loss Function
    criterion = nn.CrossEntropyLoss().to(device)
    best_score = 0
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        train_pred=[]
        train_y=[]
        model.train()
        train_loss = 0
        for img, label in tqdm(iter(train_loader)):
            
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            label = label.to(torch.int64)
            loss = criterion(logit, label)

            # backpropagation
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()/len(train_loader)
            train_pred += logit.argmax(1).detach().cpu().numpy().tolist()
            train_y += label.detach().cpu().numpy().tolist()
            
        # Evaluation Validation set
        vali_score = validation_local(model, vali_loader, device)
        
        # vali_score가 더 이상 커지지 않으면
        if scheduler is not None:
            scheduler.step(vali_score)
        
        print(f'Epoch [{epoch}] Train Score : [{accuracy_score(train_y, train_pred):.5f}] Validation Score : [{vali_score:.5f}]\n')
        
        # Model Saved
        if best_score < vali_score:
            best_score = vali_score
            torch.save(model.state_dict(), '../model/local_model.pth')
            print('Model Saved.')

In [ ]:
def validation_local(model, vali_loader, device):
    model.eval() # Evaluation
    logit_list = []
    label_list = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)
            label = label.to(torch.int64)

            logit_list.extend(model(img).argmax(1).detach().cpu().numpy().tolist())
            label_list.extend(label.detach().cpu().numpy().tolist())

    vali_f1_score = accuracy_score(label_list, logit_list)
    return vali_f1_score

In [ ]:
def predict_local(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            
            img = img.float().to(device)

            pred_logit = model(img).detach().cpu()
            model_pred.extend(pred_logit.tolist())
    return model_pred

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        img = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image=img)["image"]

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)
    
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
train_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    ToTensorV2()]
)

valid_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    ToTensorV2()]
)

test_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    ToTensorV2()]
)

In [ ]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

kfold = KFold(n_splits=splits_tr, random_state=basic_seed, shuffle=True) # CV 늘려가면서 하기
cv = np.zeros((rows_train, num_classes))
pred_test = np.zeros((rows_test, num_classes))
for n, (train_idx, val_idx) in enumerate(kfold.split(all_img_path, all_classes)):
    
    train_img_path, vali_img_path = all_img_path[train_idx], all_img_path[val_idx]
    train_classes, vali_classes = all_classes[train_idx], all_classes[val_idx]
    
    # Get Dataloader
    train_dataset = CustomDataset(train_img_path.tolist(), train_classes.tolist(), train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_classes.tolist(), train_mode=True, transforms=valid_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    model = Network().to(device)
    optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=3, factor=0.2)
    
    train_local(model, optimizer, train_loader, vali_loader, device)
    
#     checkpoint = torch.load('../model/local_model.pth')
#     model = Network().to(device)
#     model.load_state_dict(checkpoint)
    
    vali_dataset = CustomDataset(vali_img_path.tolist(), vali__classes.tolist(), train_mode=False, transforms=test_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    cv[val_idx, :] = predict_local(model, vali_loader, device)
    pred_test += np.array(predict_local(model, test_loader, device)) / splits_tr
    print(f"Fold {n+1} Score: {accuracy_score(vali__classes, cv[val_idx].argmax(axis=1))}")

print(f"Final Score: {accuracy_score(all_classes, cv.argmax(axis=1))}")

In [ ]:
pred_local_final = [class_decoder[result] for result in pred_test.argmax(1)]
with open('../pickle/pred_local_final.pickle', 'wb') as fw:
    pickle.dump(pred_local_final, fw)
test_local_df = pd.DataFrame(np.column_stack([test_img_path, pred_local_final]), columns=["path", "class"])

In [ ]:
with open('../pickle/pred_local_final.pickle', 'rb') as fw:
    pred_local_final = pickle.load(fw)
test_local_df = pd.DataFrame(np.column_stack([test_img_path, pred_local_final]), columns=["path", "class"])

In [ ]:
classList = train_y['class'].unique()
anomaly_test_dict = {}
for className in classList:
    anomaly_test_dict[className] = test_local_df[test_local_df["class"]==className]

In [ ]:
classList = train_y['class'].unique()
anomaly_dict = {}
for className in classList:
    index = train_classes[train_classes==className].index
    df = pd.DataFrame(np.column_stack([all_img_path[index], all_label[index]]), columns=["path", "label"])
    anomaly_dict[className] = df

In [ ]:
splits_tr = 4
num_classes = len(anomaly_dict['wood']['label'].unique())
basic_seed = 42
rows_train = len(anomaly_dict['wood']) # 주어진 train data의 row 수
rows_test = len(anomaly_test_dict['wood']) # 주어진 test data의 row 수

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b7', pretrained=True, num_classes=num_classes)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
train_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.RandomCrop(CFG['IMG_SIZE']-64, CFG['IMG_SIZE']-64),
#     albu.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1,
#                           rotate_limit=30, interpolation=1, border_mode=0,
#                           value=0, p=0.5),
#     albu.HorizontalFlip(p=0.2),
#     albu.VerticalFlip(p=0.2),
#     albu.RandomRotate90(p=0.2),
#     albu.CLAHE(clip_limit=2, p=0.25),
#     albu.Sharpen(p=0.25),
#     albu.RandomBrightnessContrast(brightness_limit=(-0.1, 0.1),
#                                   contrast_limit=(-0.1, 0.1), p=0.25),
#     albu.RandomResizedCrop(height=CFG['IMG_SIZE'], width=CFG['IMG_SIZE'],
#                            scale=(0.5, 1.0), ratio=(0.75, 1.3333333333333333),
#                            interpolation=1, p=1.0),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=512.0, p=1.0),
    ToTensorV2()]
)

valid_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.HorizontalFlip(p=0.5),
#     albu.VerticalFlip(p=0.5),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=512.0, p=1.0),
    ToTensorV2()]
)

test_transform = albu.Compose([
    albu.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
#     albu.Normalize(mean=IMAGENET_DEFAULT_MEAN, std=IMAGENET_DEFAULT_STD, max_pixel_value=512.0, p=1.0),
    ToTensorV2()]
)

### 모델 학습

In [ ]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.CrossEntropyLoss().to(device)
    best_score = 0
    
    for epoch in range(1,CFG["EPOCHS"]+1):
        train_pred=[]
        train_y=[]
        model.train()
#         train_loss = 0
        for img, label in tqdm(iter(train_loader)):
            
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            label = label.to(torch.int64)
            loss = criterion(logit, label)

            # backpropagation
            loss.backward()
            optimizer.step()
            
#             train_loss += loss.item()/len(train_loader)
            train_pred += logit.argmax(1).detach().cpu().numpy().tolist()
            train_y += label.detach().cpu().numpy().tolist()
            
        # Evaluation Validation set
        vali_score = validation(model, vali_loader, device)
        
        # vali_score가 더 이상 커지지 않으면
        if scheduler is not None:
            scheduler.step(vali_score)
        
        print(f'Epoch [{epoch}] Train Score : [{score_function(train_y, train_pred):.5f}] Validation Score : [{vali_score:.5f}]\n')
        
        # Model Saved
        if best_score < vali_score:
            best_score = vali_score
            torch.save(model.state_dict(), '../model/best_model.pth')
            print('Model Saved.')

In [ ]:
def validation(model, vali_loader, device):
    model.eval() # Evaluation
    logit_list = []
    label_list = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)
            label = label.to(torch.int64)

            logit_list.extend(model(img).argmax(1).detach().cpu().numpy().tolist())
            label_list.extend(label.detach().cpu().numpy().tolist())

    vali_f1_score = score_function(label_list, logit_list)
    return vali_f1_score

In [ ]:
# test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
# test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            
            img = img.float().to(device)

            pred_logit = model(img).detach().cpu()
            model_pred.extend(pred_logit.tolist())
    return model_pred

def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [ ]:
# rows_train = len(all_img_path) # 주어진 train data의 row 수
# rows_test = len(test_img_path) # 주어진 test data의 row 수
# num_trial = 100 # 파라미터 튜닝을 몇 번 진행하는지의 수
# splits_hp = 5 # 파라미터 튜닝을 진행할 때의 kfold 수
# splits_tr = 5 # 모델 트레이닝을 진행할 때의 kfold 수
# basic_seed = 42 # default seed
# num_seed_tr = 10 # 트레이닝 seed 개수
# sel_seed = 3 # 선택할 seed 개수
# num_classes = 88

# pred_dict = {}
# pred_test_dict = {}

In [ ]:
local_img_path = np.array(anomaly_dict['wood']['path'])
local_label_raw = np.array(anomaly_dict['wood']['label']).astype("int32")
local_test_path = np.array(anomaly_test_dict['wood']['path'])

test_dataset = CustomDataset(local_test_path.tolist(), None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
local_unique = sorted(np.unique(local_label_raw))
local_unique = {key:value for key,value in zip(local_unique, range(len(local_unique)))}
local_label = np.array([local_unique[k] for k in local_label_raw])

In [ ]:
CFG = {
    'IMG_SIZE':512,
    'EPOCHS':30,
    'LEARNING_RATE':5e-2,
    'BATCH_SIZE':1,
    'SEED':42
}

In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances
from lightgbm import LGBMClassifier

In [ ]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (64, 64)).flatten()
    return img

# train_imgs = [img_load(m) for m in tqdm(train_png)]
# test_imgs = [img_load(n) for n in tqdm(test_png)]

In [ ]:
local_img = np.array([img_load(m) for m in tqdm(local_img_path)])

In [ ]:
local_img.shape

In [ ]:
def lgb_objective(trial: Trial) -> float:
    params_lgb = {
        "random_state": basic_seed,
        "verbosity": -1,
        "n_estimators": 10000,
        "objective": "multiclass",
        "metric": "multi_logloss",
        'learning_rate': trial.suggest_uniform("learning_rate", 0.001, 0.01),
        "reg_alpha": trial.suggest_uniform("reg_alpha", 0.0, 1),
        "reg_lambda": trial.suggest_uniform("reg_lambda", 0.0, 1),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "num_leaves": trial.suggest_int("num_leaves", 200, 1200),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.5, 1.0), # feature_fraction
        "subsample": trial.suggest_uniform("subsample", 0.0, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 10),
        "max_bin": trial.suggest_int("max_bin", 150, 400),
    }
    
    kfold = StratifiedKFold(n_splits=splits_hp, random_state=basic_seed, shuffle = True) # Cross-validation cv=5
    cv = np.zeros((rows_train, num_classes))

    for n, (train_idx, val_idx) in enumerate(kfold.split(local_img, local_label)):

        x_train, x_val = local_img[train_idx], local_img[val_idx]
        y_train, y_val = local_label[train_idx], local_label[val_idx]

        lgbmodel = LGBMClassifier(**params_lgb)
                                                                                        # 진행상황 보고싶을때 -1을 100으로
        lgbmodel.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=-1, eval_metric=custom_metric) 
        cv[val_idx, :] = lgbmodel.predict_proba(x_val)
        
#         print('fold', n+1, 'f1_score:', score_function(y_val, cv[val_idx, :].argmax(1)))
        
#     print('f1_score:', score_function(local_label, np.argmax(cv, axis=1)))
    return score_function(local_label, np.argmax(cv, axis=1))

In [ ]:
num_trial = 100
splits_hp = 4

In [ ]:
def custom_metric(truth, predictions):  
    pred_labels = predictions.reshape(len(np.unique(truth)),-1).argmax(axis=0)
    f1 = f1_score(truth, pred_labels, average='macro')
    return ('macroF1', f1, True) 

In [ ]:
sampler = TPESampler(seed=basic_seed)
lgb_study = optuna.create_study(study_name="lgb_parameter_opt", direction="maximize", sampler=sampler)
lgb_study.optimize(lgb_objective, n_trials=num_trial)

lgb_best_hyperparams = lgb_study.best_trial.params
lgb_base_hyperparams = {'objective':'multiclass', 'n_estimators':10000,
                        'lambda_l1':lgb_best_hyperparams['reg_alpha'],
                        'lambda_l2':lgb_best_hyperparams['reg_lambda'],
                        'reg_alpha':None, 'reg_lambda':None
                       }
lgb_best_hyperparams.update(lgb_base_hyperparams)
print("The best hyperparameters are:\n", lgb_best_hyperparams)

In [ ]:
kfold = KFold(n_splits=splits_tr, random_state=basic_seed, shuffle=True) # CV 늘려가면서 하기
cv = np.zeros((rows_train, num_classes))
pred_test = np.zeros((rows_test, num_classes))
for n, (train_idx, val_idx) in enumerate(kfold.split(local_img_path, local_label)):
    
    train_img_path, vali_img_path = local_img_path[train_idx], local_img_path[val_idx]
    train_label, vali_label = local_label[train_idx], local_label[val_idx]
    
    # Get Dataloader
    train_dataset = CustomDataset(train_img_path.tolist(), train_label.tolist(), train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=True, transforms=valid_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    model = Network().to(device)
    optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=5, factor=0.2)
    
    train(model, optimizer, train_loader, vali_loader, scheduler, device)
    
    checkpoint = torch.load('../model/best_model.pth')
    model = Network().to(device)
    model.load_state_dict(checkpoint)
    
    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=False, transforms=test_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    cv[val_idx, :] = predict(model, vali_loader, device)
    pred_test += np.array(predict(model, test_loader, device)) / splits_tr
    print(f"Fold {n+1} Score: {score_function(vali_label, cv[val_idx].argmax(axis=1))}")
    
pred_dict['eff_b0'+str(seed)] = cv
pred_test_dict['eff_b0'+str(seed)] = pred_test

In [ ]:
pred_dict = {}
pred_test_dict = {}

In [ ]:
pred_dict['eff_b0'+str(42)] = cv
pred_test_dict['eff_b0'+str(42)] = pred_test

In [ ]:
kfold = KFold(n_splits=splits_tr, random_state=basic_seed, shuffle=True) # CV 늘려가면서 하기
cv = np.zeros((rows_train, num_classes))
pred_test = np.zeros((rows_test, num_classes))
for n, (train_idx, val_idx) in enumerate(kfold.split(all_img_path, all_label)):
    
    train_img_path, vali_img_path = all_img_path[train_idx], all_img_path[val_idx]
    train_label, vali_label = all_label[train_idx], all_label[val_idx]
    
    # Get Dataloader
    train_dataset = CustomDataset(train_img_path.tolist(), train_label.tolist(), train_mode=True, transforms=train_transform)
    train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=True, transforms=valid_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    model = Network().to(device)
    optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='max', patience=3, factor=0.2)
    
    train(model, optimizer, train_loader, vali_loader, scheduler, device)
    
    checkpoint = torch.load('../model/best_model.pth')
    model = Network().to(device)
    model.load_state_dict(checkpoint)
    
    vali_dataset = CustomDataset(vali_img_path.tolist(), vali_label.tolist(), train_mode=False, transforms=test_transform)
    vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)
    
    cv[val_idx, :] = predict(model, vali_loader, device)
    pred_test += np.array(predict(model, test_loader, device)) / splits_tr
    print(f"Fold {n+1} Score: {score_function(vali_label, cv[val_idx].argmax(axis=1))}")
    
pred_dict['eff_b0'+str(seed)] = cv
pred_test_dict['eff_b0'+str(seed)] = pred_test

### 추론

### 제출물 생성

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}
pred_final = [label_decoder[result] for result in pred_test.argmax(1)]

In [ ]:
from datetime import datetime
submission = pd.read_csv('../data/sample_submission.csv')
submission["label"] = pred_final
submission_time = datetime.today().strftime('%Y-%m-%d-%M-%S')
submission.to_csv(f'../submission/{submission_time}.csv', index = False)